<h1 id="tocheading">XGB Troubleshooting</h1>
<div id="toc"></div>

The purpose of this notebook is to show some comfusion we encoucotered when applying XGboost. We used same data for binary prediction and applied same technique(SMOTE) to handle imbalance. And tried to keep all the hyperparameters the same as well. But still the result is different.  
AUC from SPSS output is .602 vs. .501 in the notebook. I have not checked xgb script in Modeler. Just wondering if  went wrong here.

Unexpected inconsistency:
1. AUC: SPSS(.602) vs. Notebook(.501 or .559)
2. The built-in evaluation produced higher AUC values than roc_auc_score() on test set.

** Any ideas?? **

Note: 
1. Run the following cell to view the full Table of Contents:)
2. I am experiencing trouble creating a new project in DSX right now. If you need to check the datasets(both original and processed data from Modeler), see [here](https://github.com/CatherineCao2016/Datasets). 

In [35]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')


<IPython.core.display.Javascript object>

## Imports

In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [2]:
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_79f48b7520e04b748bcd13ec11f1f911(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_e477f3c5a924bd596600a5b14fe82d48597c4447','domain': {'id': '2028e4c0200e41428802c11a8d39a2a6'},
            'password': 'ieP[&WmPw{96IBww'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df_data_1 = pd.read_csv(get_object_storage_file_with_credentials_79f48b7520e04b748bcd13ec11f1f911('Defaultproject', 'History_Transactions_v4.csv')).drop("Hours Since Last Transaction", 1)
df_data_1 = df_data_1[['Dollar_Amount', 
                       'Transaction_Type',
                       'Store_Type', 
                       'Cardholder_Region', 
                       'Country',
                       'Fraudulent',
                       'Last3hourTransactions',
                       'Hours_Since_Last_Transaction'
                      ]]
df_data_1.head()

,Dollar_Amount,Transaction_Type,Store_Type,Cardholder_Region,Country,Fraudulent,Last3hourTransactions,Hours_Since_Last_Transaction
0,188.56,swiped,ATM Withdrawal,E,USA,F,4,21
1,160.54,keyed,Grocery,NE,Australia,F,3,21
2,153.24,swiped,Retail - Pharmacy,SW,European Union,T,2,21
3,148.66,keyed,Communications - telephone and wireless,NW,USA,F,1,35
4,174.03,keyed,Spa/Beauty,SW,Asia,F,1,25


## Split into Training and Test

In [4]:
y = df_data_1['Fraudulent'].map(lambda l: 1 if l == 'T' else 0)
X = df_data_1.drop(['Fraudulent'], axis=1)

from sklearn.cross_validation import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234567)

print("Number of training records: " + str(len(X_train)))
print("Number of testing records : " + str(len(X_test)))

Number of training records: 44028
Number of testing records : 11007


/gpfs/fs01/user/s34f-24c5585e483e16-c0ce2641d0ef/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Handle Imbalance using SMOTE(only on Training set)

In [5]:
# restart kernel if see error
from imblearn.over_sampling import SMOTE

In [6]:
X_train_dummy = pd.get_dummies(X_train)
sm = SMOTE(random_state=2662761, ratio = 'auto', k_neighbors = 5)
X_train_res, y_train_res = sm.fit_sample(X_train_dummy, y_train)


print('Fraud/Non-Fraud Ratio Before SMOTE: ' + str(y_train.sum()/len(y_train)))
print('Fraud/Non-Fraud Ratio After SMOTE: ' + str(y_train_res.sum()/len(y_train_res)))

print("Fraud in Training after SMOTE = T: " + str(sum(y_train_res)))
print("Fraud in Training after SMOTE = F: " + str(len(y_train_res) - sum(y_train_res)))
print("\n************Test Set************\n")
print("Number of testing records : " + str(len(X_test)))
print("Training: T = : " + str(sum(y_test)))
print("Training: F = :" + str(len(y_test) - sum(y_test)))

Fraud/Non-Fraud Ratio Before SMOTE: 0.05233033524121014
Fraud/Non-Fraud Ratio After SMOTE: 0.5
Fraud in Training after SMOTE = T: 41724
Fraud in Training after SMOTE = F: 41724

************Test Set************

Number of testing records : 11007
Training: T = : 599
Training: F = :10408


## XGboost

The hyperparameters are set as following, the same as Modeler's setting.

In [7]:
xgb_model = XGBClassifier(
    tree_method= "auto",
    n_estimators = 10,
    max_depth = 6,
    min_child_weight = 1.0,
    max_delta_step = 0.0,
    objective = "binary:logistic",
    seed = 6924827,
    subsample = 1.0,
    learning_rate = .3,
    gamma = 0.0,
    colsample_bytree = 1.0,
    colsample_bylevel = 1.0,
    reg_lambda = 1.0,
    reg_alpha = 0.0,
    scale_pos_weight = 1.0)

In [8]:
xgb_model.fit(X_train_res, y_train_res, eval_set = [(X_train_res, y_train_res), (pd.get_dummies(X_test).as_matrix(), y_test)], eval_metric = "auc", verbose = True)

[0]	validation_0-auc:0.893968	validation_1-auc:0.641551
[1]	validation_0-auc:0.916541	validation_1-auc:0.649042
[2]	validation_0-auc:0.947808	validation_1-auc:0.65575
[3]	validation_0-auc:0.956587	validation_1-auc:0.653419
[4]	validation_0-auc:0.967039	validation_1-auc:0.647911
[5]	validation_0-auc:0.969599	validation_1-auc:0.65116
[6]	validation_0-auc:0.972861	validation_1-auc:0.654913
[7]	validation_0-auc:0.975912	validation_1-auc:0.652043
[8]	validation_0-auc:0.977395	validation_1-auc:0.651779
[9]	validation_0-auc:0.978747	validation_1-auc:0.655097


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
       colsample_bytree=1.0, gamma=0.0, learning_rate=0.3,
       max_delta_step=0.0, max_depth=6, min_child_weight=1.0, missing=None,
       n_estimators=10, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.0,
       reg_lambda=1.0, scale_pos_weight=1.0, seed=6924827, silent=True,
       subsample=1.0, tree_method='auto')

In [9]:
pred_classifier = xgb_model.predict(pd.get_dummies(X_test).as_matrix())

/gpfs/fs01/user/s34f-24c5585e483e16-c0ce2641d0ef/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [10]:
roc = roc_auc_score(y_test, pred_classifier, )
print("ROC: " +  str(roc))

ROC: 0.5005945246946294


## Using SMOTE output from SPSS

To control the randomness from Train/Test split and SMOTE, we tried using the SMOTE output from SPSS directly.

In [14]:
SMOTEoutput = pd.read_csv(get_object_storage_file_with_credentials_79f48b7520e04b748bcd13ec11f1f911('Defaultproject', 'modeler_smoteout.csv'))
SMOTEoutput = SMOTEoutput.drop(['Account_Number','Transaction_Time', 'last transaction time', 'Hours Since Last Transaction', 'Transaction_Dates',	'Last_Trasaction_Dates'], 1)


SMOTEoutput.head()

,Dollar_Amount,Transaction_Type,Store_Type,Cardholder_Region,Country,Fraudulent,Last3hourTransactions,Hours_Since_Last_Transaction,Partition
0,188.56,swiped,ATM Withdrawal,E,USA,F,4,21,1_Training
1,160.54,keyed,Grocery,NE,Australia,F,3,21,1_Training
2,153.24,swiped,Retail - Pharmacy,SW,European Union,T,2,21,1_Training
3,148.66,keyed,Communications - telephone and wireless,NW,USA,F,1,35,2_Testing
4,174.03,keyed,Spa/Beauty,SW,Asia,F,1,25,1_Training


In [22]:
train = SMOTEoutput[SMOTEoutput.Partition == "1_Training"].drop('Partition', 1)
test = SMOTEoutput[SMOTEoutput.Partition == "2_Testing"].drop('Partition', 1)

y = train['Fraudulent'].map(lambda l: 1 if l == 'T' else 0)
X = pd.get_dummies(train.drop(['Fraudulent'], axis=1))

y_test_spss = test['Fraudulent'].map(lambda l: 1 if l == 'T' else 0)
X_test_spss = pd.get_dummies(test.drop(['Fraudulent'], axis=1))

In [24]:
xgb_model2 = XGBClassifier(
    tree_method= "auto",
    n_estimators = 10,
    max_depth = 6,
    min_child_weight = 1.0,
    max_delta_step = 0.0,
    objective = "binary:logistic",
    seed = 6924827,
    subsample = 1.0,
    learning_rate = .3,
    gamma = 0.0,
    colsample_bytree = 1.0,
    colsample_bylevel = 1.0,
    reg_lambda = 1.0,
    reg_alpha = 0.0,
    scale_pos_weight = 1.0)
xgb_model2.fit(X.as_matrix(), y, eval_set = [(X.as_matrix(), y), (X_test_spss.as_matrix(), y_test_spss)], eval_metric = "auc", verbose = True)

[0]	validation_0-auc:0.741852	validation_1-auc:0.600351
[1]	validation_0-auc:0.75262	validation_1-auc:0.599793
[2]	validation_0-auc:0.759496	validation_1-auc:0.606255
[3]	validation_0-auc:0.762706	validation_1-auc:0.608371
[4]	validation_0-auc:0.766306	validation_1-auc:0.607905
[5]	validation_0-auc:0.771224	validation_1-auc:0.609184
[6]	validation_0-auc:0.775101	validation_1-auc:0.60665
[7]	validation_0-auc:0.778528	validation_1-auc:0.600136
[8]	validation_0-auc:0.781652	validation_1-auc:0.598593
[9]	validation_0-auc:0.784641	validation_1-auc:0.599681


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
       colsample_bytree=1.0, gamma=0.0, learning_rate=0.3,
       max_delta_step=0.0, max_depth=6, min_child_weight=1.0, missing=None,
       n_estimators=10, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.0,
       reg_lambda=1.0, scale_pos_weight=1.0, seed=6924827, silent=True,
       subsample=1.0, tree_method='auto')

In [25]:
pred_classifier_spssoutput = xgb_model2.predict(X_test_spss.as_matrix())

/gpfs/fs01/user/s34f-24c5585e483e16-c0ce2641d0ef/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [28]:
roc2 = roc_auc_score(y_test_spss, pred_classifier_spssoutput)
print("ROC: " +  str(roc2))

ROC: 0.5586843324628064


Still the ROC is different. 

## Appendix

Some screenshots from SPSS:
<img src="https://github.com/CatherineCao2016/pics/raw/master/xgb_modeler_1.png" width="500" height="200" align="middle"/>
<img src="https://github.com/CatherineCao2016/pics/raw/master/xgb_modeler_2.png" width="400" height="200" align="middle"/>
<img src="https://github.com/CatherineCao2016/pics/raw/master/xgb_modeler_3.png" width="400" height="200" align="middle"/>
<img src="https://github.com/CatherineCao2016/pics/raw/master/xgb_modeler_4.png" width="400" height="200" align="middle"/>
<img src="https://github.com/CatherineCao2016/pics/raw/master/xgb_modeler_5.png" width="400" height="200" align="middle"/>